# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 11 2023 9:36AM,254749,10,PRF-44162-BO1,Bio-PRF,Released
1,Jan 11 2023 9:36AM,254749,10,PRF-46128-BO,Bio-PRF,Released
2,Jan 11 2023 9:36AM,254749,10,PRF-46384,Bio-PRF,Released
3,Jan 11 2023 9:36AM,254749,10,PRF-46388,Bio-PRF,Released
4,Jan 11 2023 9:36AM,254749,10,PRF-46391,Bio-PRF,Released
5,Jan 11 2023 9:36AM,254749,10,PRF-46394,Bio-PRF,Released
6,Jan 11 2023 9:36AM,254749,10,PRF-46399,Bio-PRF,Released
7,Jan 11 2023 9:36AM,254749,10,PRF-46405,Bio-PRF,Released
8,Jan 11 2023 9:36AM,254749,10,PRF-46409,Bio-PRF,Released
9,Jan 11 2023 9:36AM,254749,10,PRF-46412,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,254746,Released,16
31,254747,Released,1
32,254748,Released,3
33,254749,Released,14
34,254750,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254746,NaN,NaN,16.0
254747,NaN,NaN,1.0
254748,NaN,NaN,3.0
254749,NaN,NaN,14.0
254750,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254650,0.0,0.0,18.0
254655,0.0,1.0,0.0
254657,0.0,0.0,1.0
254668,0.0,1.0,0.0
254673,8.0,3.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254650,0,0,18
254655,0,1,0
254657,0,0,1
254668,0,1,0
254673,8,3,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254650,0,0,18
1,254655,0,1,0
2,254657,0,0,1
3,254668,0,1,0
4,254673,8,3,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254650,,,18
1,254655,,1,
2,254657,,,1
3,254668,,1,
4,254673,8,3,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 11 2023 9:36AM,254749,10,Bio-PRF
14,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix
15,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc."
18,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.
19,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation
35,Jan 11 2023 9:19AM,254745,10,Emerginnova
38,Jan 11 2023 9:19AM,254744,16,TASA USA Inc.
39,Jan 11 2023 9:19AM,254743,10,"Methapharm, Inc."
53,Jan 11 2023 9:18AM,254739,10,ISDIN Corporation
116,Jan 11 2023 9:18AM,254738,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 11 2023 9:36AM,254749,10,Bio-PRF,,,14
1,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,,,1
2,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc.",,,3
3,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.,,,1
4,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation,,,16
5,Jan 11 2023 9:19AM,254745,10,Emerginnova,,,3
6,Jan 11 2023 9:19AM,254744,16,TASA USA Inc.,,,1
7,Jan 11 2023 9:19AM,254743,10,"Methapharm, Inc.",,,14
8,Jan 11 2023 9:18AM,254739,10,ISDIN Corporation,,,63
9,Jan 11 2023 9:18AM,254738,10,ISDIN Corporation,,,20


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 9:36AM,254749,10,Bio-PRF,14,,
1,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,1,,
2,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc.",3,,
3,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.,1,,
4,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation,16,,
5,Jan 11 2023 9:19AM,254745,10,Emerginnova,3,,
6,Jan 11 2023 9:19AM,254744,16,TASA USA Inc.,1,,
7,Jan 11 2023 9:19AM,254743,10,"Methapharm, Inc.",14,,
8,Jan 11 2023 9:18AM,254739,10,ISDIN Corporation,63,,
9,Jan 11 2023 9:18AM,254738,10,ISDIN Corporation,20,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 9:36AM,254749,10,Bio-PRF,14,,
1,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,1,,
2,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc.",3,,
3,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.,1,,
4,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation,16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 9:36AM,254749,10,Bio-PRF,14.0,NaN,NaN
1,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc.",3.0,NaN,NaN
3,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.,1.0,NaN,NaN
4,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation,16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 9:36AM,254749,10,Bio-PRF,14.0,0.0,0.0
1,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc.",3.0,0.0,0.0
3,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.,1.0,0.0,0.0
4,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation,16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4585157,177.0,7.0,8.0
15,509348,19.0,0.0,0.0
16,1273641,5.0,0.0,0.0
19,1528272,46.0,1.0,0.0
21,509312,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4585157,177.0,7.0,8.0
1,15,509348,19.0,0.0,0.0
2,16,1273641,5.0,0.0,0.0
3,19,1528272,46.0,1.0,0.0
4,21,509312,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,177.0,7.0,8.0
1,15,19.0,0.0,0.0
2,16,5.0,0.0,0.0
3,19,46.0,1.0,0.0
4,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,177.0
1,15,Released,19.0
2,16,Released,5.0
3,19,Released,46.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,8.0,0.0,0.0,0.0,0.0
Executing,7.0,0.0,0.0,1.0,1.0
Released,177.0,19.0,5.0,46.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,0.0,1.0,1.0
2,Released,177.0,19.0,5.0,46.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,0.0,1.0,1.0
2,Released,177.0,19.0,5.0,46.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()